In [18]:
import requests as rq
import json
import pandas as pd
from pandas import DataFrame as df
import datetime
import csv
import numpy as np
import vectorbt as vbt
import finlab_crypto
import time
import signal
from function import RetrieveData as RD
from function import calulate as cl



finlab_crypto.setup()
ohlcv_4h = finlab_crypto.crawler.get_all_binance('BTCUSDT', '4h')
ohlcv=finlab_crypto.crawler.get_all_binance('BTCUSDT', '1m')


import talib #計算所需RSI指標週期
rsi6=talib.RSI(ohlcv_4h.close,timeperiod=6)
rsi12=talib.RSI(ohlcv_4h.close,timeperiod=12)
rsi24=talib.RSI(ohlcv_4h.close,timeperiod=24)
rsi48=talib.RSI(ohlcv_4h.close,timeperiod=48)
rsi96=talib.RSI(ohlcv_4h.close,timeperiod=96)
def IRR(x ,current_price=ohlcv.open[-1]):
    """
    計算內部報酬率 (IRR)

    x: 交易數據
    current_price: 當前價格，默認為最新的開盤價
    """
    amount = []
    price = []

    open = ohlcv.open
    amount.extend(x.result)
    price.extend(x.time)
    
    for i in range(len(price)):
        price[i] = price[i] - (price[i] % 60)
        price[i] = open.loc[str(np.datetime64(price[i], 's'))]
    
    # 計算未實現損益
    un = x.loc[x.result > 0].result.sum() + x.loc[x.result < 0].result.sum()
    un = un * current_price
    amount = [p * -a for p, a in zip(price, amount)]
    amount.append(un)
    
    import numpy_financial as npf

    # 計算 IRR
    irr = npf.irr(amount)
    print(f'Internal Rate of Return (IRR): {irr:.2%}')
    return irr

# import importlib
# reloaded_module = importlib.reload(__import__('total_return'))

All caught up..!
All caught up..!


In [27]:
columns=['address', '交易次數', 'IRR',  
       'buy_rsi6', 'buy_rsi12', 'buy_rsi24','buy_rsi48', 'buy_rsi96','sell_rsi6',  'sell_rsi12', 'sell_rsi24', 'sell_rsi48', 'sell_rsi96',
       'oversell_ratio6','overbought_ratio6', 
       'oversell_ratio12','overbought_ratio12',
       'oversell_ratio24', 'overbought_ratio24',
       'oversell_ratio48', 'overbought_ratio48',
       'oversell_ratio96', 'overbought_ratio96',
       'buy_value', 'sell_value',
       'buying_time', 'selling_time',  'download']
#分析所需欄位
df = pd.DataFrame(columns=columns)


In [20]:
"""
取得所需要的地址，持續執行到500個地址
"""

RootAddress="bc1q3e59jtjgjr97y9tzsl2d8pf3e9atckmj2wsnzk" 
df.loc[0,"address"]=RootAddress
addr_list=RD.extract_address(RootAddress)
df=RD.check_and_added_address(addr_list,df)


https://blockchain.info/rawaddr/bc1q3e59jtjgjr97y9tzsl2d8pf3e9atckmj2wsnzk?limit=5000&offset=0
重複
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功
加入成功


In [ ]:
from function import RetrieveData as RD
"""
示範如何抓一筆Address的data
1.取得Address名稱
2.產生需要的URL
3.取得資料
後續把每筆資料跑完，不能用迴圈，API會擋
"""
address=df.address[0]
RD.get_url(address)
RD.get_data(address)



In [ ]:
"""
指標計算，跑回圈計算取得所有欄位，
"""
df['address'] = target['address'] #target存了之前抓的資料，這邊直接把地址asign過來，後續計算只要替換參數跑完回權就能很快算出來
from function import calulate
for count in range(0,len(df)):
    x=RD.read_data(count,target) #檔案都已下載到本地了
    if x is not None:

        try:
            #計算資料
            df.loc[count,'交易次數']=len(x)
            df.loc[count,'IRR']=IRR(x,ohlcv_4h.open.rolling(24).mean().iloc[-1])#計算報酬率
            cl.count_BandS(x,target=df,count=count)#計算買進與賣出次數
            cl.total_value(x,count,df)#買賣的交易數量
            cl.計算加權RSI(rsi6,x,count,df,'6')
            cl.計算加權RSI(rsi12,x,count,df,'12')
            cl.計算加權RSI(rsi24,x,count,df,'24')
            cl.計算加權RSI(rsi48,x,count,df,'48')
            cl.計算加權RSI(rsi96,x,count,df,'96')    
            cl.oversold_v2(rsi6,x,count,df,column="oversell_ratio6",zone=30)
            cl.overbought_v2(rsi6,x,count,df,column="overbought_ratio6",zone=70)
            cl.oversold_v2(rsi12,x,count,df,column="oversell_ratio12",zone=30)
            cl.overbought_v2(rsi12,x,count,df,column="overbought_ratio12",zone=70)  
            cl.oversold_v2(rsi24,x,count,df,column="oversell_ratio24",zone=30)
            cl.overbought_v2(rsi24,x,count,df,column="overbought_ratio24",zone=70)
            cl.oversold_v2(rsi48,x,count,df,column="oversell_ratio48",zone=30)            
            cl.overbought_v2(rsi48,x,count,df,column="overbought_ratio48",zone=70)
            cl.oversold_v2(rsi96,x,count,df,column="oversell_ratio96",zone=30)
            cl.overbought_v2(rsi96,x,count,df,column="overbought_ratio96",zone=70)
   
        except Exception as e:
            print(f'计算失败: {e}')
    else:
        print(f"檔案下載失敗{target.loc[count,'download']}")



In [29]:
#計算結果
df

,address,交易次數,IRR,buy_rsi6,buy_rsi12,buy_rsi24,buy_rsi48,buy_rsi96,sell_rsi6,sell_rsi12,...,overbought_ratio24,oversell_ratio48,overbought_ratio48,oversell_ratio96,overbought_ratio96,buy_value,sell_value,buying_time,selling_time,download
0,bc1q3e59jtjgjr97y9tzsl2d8pf3e9atckmj2wsnzk,47,-0.001366,59.094192,58.769179,59.321785,60.876378,61.395317,60.65375,59.832582,...,-0.0,0.0,-0.0,0.0,-0.0,16607606,-16607606,24,23,NaN
1,bc1q6qud26duqwucxv36pvwcakxlw7vwryyghx78xm,179,-0.001309,64.131898,64.037835,63.760096,62.927151,61.317646,63.30617,63.540324,...,0.319131,0.0,0.189091,0.0,0.053076,70735220,-70735220,95,84,NaN
2,bc1qepz9ymljjdp4afahzmlt3w706pvwxc4uva5ha6,442,-0.003423,56.733018,57.299265,57.566388,56.675627,54.625432,56.639003,57.605357,...,0.175095,0.007567,0.085713,0.0,0.032235,906208647,-905180401,106,336,NaN
3,3KdtD1yda8x76Mk2K8WZriBVa95pFHhHsF,50,0.005308,51.214473,52.135169,54.302009,54.908047,54.444212,48.689458,48.683966,...,0.024851,0.0,-0.0,0.0,-0.0,42998401,-42998401,25,25,NaN
4,18hoC6dPxhqUH7c71uSahy55eYmN9v7Y38,288,-0.002858,50.690468,50.811699,50.785818,50.562028,50.41144,51.353189,51.07045,...,0.03706,0.0,0.018174,0.0,-0.0,6409880170,-6409880170,145,143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,3DHX68pCGdY5Hr6rVG7A5KcAbesxuWs5CW,64,0.000922,48.969978,48.01711,47.303641,46.854348,46.8838,48.348535,47.696393,...,-0.0,0.031032,-0.0,0.0,-0.0,1645507,-1645507,32,32,NaN
503,39e6qb4crSvr7bX9L9tKsKTKEEPeGpzkqf,225,-0.003714,48.198074,49.457622,49.848455,49.788156,49.509238,52.274421,54.009636,...,0.082006,0.0,0.036331,0.0,0.018229,119462409,-119462409,114,111,NaN
504,3NpwktSMLfeD4dA1NxQFxwoErqVaxNYBqX,8,-0.131888,46.96243,52.296606,52.731524,50.949505,49.775,49.17509,46.443897,...,-0.0,0.0,-0.0,0.0,-0.0,2676253,-2676253,4,4,NaN
505,31vMqsMrDa8ESPb6CyaTh1hvUwfTmhRPeo,160,0.002206,47.381563,46.85957,46.29609,46.763992,47.828559,42.10998,44.169126,...,0.046043,0.0,0.011142,0.0,0.010503,257833543,-257833543,81,79,NaN
